<a href="https://colab.research.google.com/github/tluxxx/weekly-pattern-in-stock-markets/blob/main/weekly_patterns_(part_1_16_week_cycle_with_pattern_according_to_Gebert).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploiting weekly patterns in stock market for trading
#### part 1: testing a trading pattern in a 16 weeks cycle

This series is based on a strategy-idea originally generated by Thomas Gebert (Kurzfrist-Strategien fuer Anleger, Boersenbuchverlag, 2019 (in German)). The original strategy was tested and expanded. Further modifications are investigated. The this part contains:


1.   Testing of Original Strategy during the years [2020-2018]
2.   Expanding of Original Strategy to the years [2019-2024]
1.   Senstivity testing

#1. Preparation, Data Upload and Definition of Helper Functions

In [ ]:
# preparation
from google.colab import drive
drive.mount("/content/gdrive")

In [4]:
# importing all modules
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go

In [9]:
# direct download price dataframe from yfinance
end_date = '2024-03-30'
start_date = '2000-01-01'
price = yf.download('^GDAXI', start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


# 2. Helper Functions Code

In [29]:
# preparation: resampling daily to weekly data, numbering weeks (original strategy (first trading day of the week at Close)
def transforming_daily_weekly_close(pr):
  pr['date'] = pr.index
  df_input = {'start_w': pr.date.resample('W').first(),
              'price': pr.Close.resample('W').first()}
  pr_w = pd.DataFrame(df_input)
  pr_w = pr_w.assign(week_nb=range(len(pr_w)))
  pr_w.set_index('start_w', inplace=True)
  return pr_w

# preparation: resampling daily to weekly data, numbering weeks (modified strategy (first trading day of the week at Open)
def transforming_daily_weekly_open(pr):
  pr['date'] = pr.index
  df_input = {'start_w': pr.date.resample('W').first(),
              'price': pr.Open.resample('W').first()}
  pr_w = pd.DataFrame(df_input)
  pr_w = pr_w.assign(week_nb=range(len(pr_w)))
  pr_w.set_index('start_w', inplace=True)
  return pr_w

# preparation: matching weekly date with weekly pattern in a cycle
def pos_weekly(pr_w, pos, shift):
  pos_dict = dict(enumerate(pos))
  # calculation of week-type and resulting positions depending on the shift
  pr_w['week_type'] = (pr_w['week_nb'] + shift) % len(pos)
  pr_w['pos'] = pr_w['week_type'].map(pos_dict)


In [23]:
# analyses: calculation of pnl for a given weekly pattern (including positions)
def pnl_calculation(pr_w):
  res = np.ones(3)
  # calculation of PnL ** strategy
  rets = pr_w['price'].pct_change()*pr_w['pos'].shift(1)
  pnl_strategy = (1 + rets).cumprod()
  pnl_strategy2 = rets.cumsum() + 1
  # calculation of PnL ** buy and hold
  pnl_buyhold = (1 + pr_w['price'].pct_change()).cumprod()
  # generation of results dataframe
  input = {'pnl_strategy':pnl_strategy, 'pnl_strategy2':pnl_strategy2, 'pnl_b&h':pnl_buyhold}
  pnl = pd.DataFrame(input)
  # calculating the final results
  res[0] = pnl['pnl_b&h'][-1]
  res[1] = pnl['pnl_strategy'][-1]
  res[2] = pnl['pnl_strategy2'][-1]
  return pnl, res

# plotting: pnl over time
def pnl_plotting(pnl, title):
  fig = px.line(pnl, x=pnl.index, y=['pnl_strategy','pnl_strategy2','pnl_b&h'])
  fig.update_layout(title=title)
  fig.update_layout(autosize=False,width=800,height=400)
  fig.update_layout(template = 'plotly_dark', xaxis_title='Date', yaxis_title='PnL')
  fig.show()

# 3. Original Strategy by Gebert (slightly simplified)
### Features:  
*   16-weeks cycle
*   tradetime for opening positions (buy/short) & closing positions (sell/cover): each monday 17:30 (at close)

### Modifications:
*   Original Strategy: If first trading day of a week is not Monday, trading on the last trading day of the previous week
*   Proposed Simplification: if first trading day of a week is not Monday, trading on the first trading day of that particular week


In [11]:
# original gebert pattern and setting other parameters
pos_original = [0, 0, 0, 0, 0, 0, 0, -1, 0, 0, -1, 0, 1, 1, 1, -1]
shift_weeks = 12
pos = pos_original
# transforming daily into weekly data and calculting weekly trading positions
price_w = transforming_daily_weekly_close(price)
pos_weekly(price_w, pos, shift_weeks)

In [12]:
# check if 2018-11-12 is in week_type 4
price_w[price_w.index >'2018-11-01'].head(3)

,price,week_nb,week_type,pos
start_w,,,,
2018-11-05,11494.959961,983,3,0
2018-11-12,11325.440430,984,4,0
2018-11-19,11244.540039,985,5,0


In [24]:
# calculation and plottint/printing of pnl time-series and final results for the [2000-2018] period
price_w_original = price_w[price_w.index < '2018-12-31']
pnl_original, res_original = pnl_calculation(price_w_original)
title='Equity Curve for 16-week-cycle (orig.pattern) vs. Buy & Hold Instrument: DAX [2000-2018]'
pnl_plotting(pnl_original, title)
print(f'Buy and Hold:   {res_original[0]:.2f}   Strategy:  {res_original[1]:.2f}    Strategy2:  {res_original[2]:.2f} ')

Buy and Hold:   1.54   Strategy:  23.56    Strategy2:  4.36 


In [26]:
# calculation and plottint/printing of pnl time-series and final results for the [2000-2024] period
pnl_all, res_all = pnl_calculation(price_w)
title='Equity Curve for 16-week-cycle (orig.pattern) vs. Buy & Hold Instrument: DAX [2000-2024]'
pnl_plotting(pnl_all, title)
print(f'Buy and Hold:   {res_all[0]:.2f}   Strategy:  {res_all[1]:.2f}    Strategy2:  {res_all[2]:.2f} ')

Buy and Hold:   2.71   Strategy:  26.83    Strategy2:  4.51 


In [25]:
# calculation and plotting/printing of pnl time-series and final results for the [2019-2024] period
# (i.e. after the publication in 2019)
price_w_recent = price_w[price_w.index>'2019-01-01']
pnl_recent, res_recent = pnl_calculation(price_w_recent)
title = 'Equity Curve for 16-week-cycle (orig.pattern) vs. Buy & Hold Instrument: DAX [2019-2024]'
pnl_plotting(pnl_recent, title)
print(f'Buy and Hold:   {res_recent[0]:.2f}   Strategy:  {res_recent[1]:.2f}    Strategy2:  {res_recent[2]:.2f} ')

Buy and Hold:   1.73   Strategy:  1.16    Strategy2:  1.18 


# 4. Modification on modified Original Strategy

*   identical to the slightly modified Original Strategy of Gebert (except tradetime)
*   tradetime for opening positions (buy/short) & closing positions (sell/cover): each monday 9:00 (at open)



In [30]:
# transforming daily into weekly data and calculting weekly trading positions
price_w = transforming_daily_weekly_open(price)
pos_weekly(price_w, pos, shift_weeks)

In [31]:
# check if 2018-11-12 is in week_type 4
price_w[price_w.index >'2018-11-01'].head(3)

,price,week_nb,week_type,pos
start_w,,,,
2018-11-05,11522.330078,983,3,0
2018-11-12,11591.589844,984,4,0
2018-11-19,11384.000000,985,5,0


In [32]:
# calculation and plottint/printing of pnl time-series and final results for the [2000-2024] period
pnl_all, res_all = pnl_calculation(price_w)
title='Equity Curve for 16-week-cycle (orig.pattern) mod.system vs. Buy & Hold Instrument: DAX [2000-2024]'
pnl_plotting(pnl_all, title)
print(f'Buy and Hold:   {res_all[0]:.2f}   Strategy:  {res_all[1]:.2f}    Strategy2:  {res_all[2]:.2f} ')

Buy and Hold:   2.62   Strategy:  31.26    Strategy2:  4.66 


In [33]:
# calculation and plotting/printing of pnl time-series and final results for the [2019-2024] period
# (i.e. after the publication in 2019)
price_w_recent = price_w[price_w.index>'2019-01-01']
pnl_recent, res_recent = pnl_calculation(price_w_recent)
title = 'Equity Curve for 16-week-cycle (orig.pattern) vs. Buy & Hold Instrument: DAX [2019-2024]'
pnl_plotting(pnl_recent, title)
print(f'Buy and Hold:   {res_recent[0]:.2f}   Strategy:  {res_recent[1]:.2f}    Strategy2:  {res_recent[2]:.2f} ')

Buy and Hold:   1.74   Strategy:  1.36    Strategy2:  1.34 


In [35]:
# export of results to excel file
with pd.ExcelWriter(r'/content/gdrive/MyDrive/Colab Notebooks/Weekly Pattern/16_weeks_cycle (data).xlsx') as writer:
     price.to_excel(writer, sheet_name="daily data")
     price_w.to_excel(writer, sheet_name="weekly data")
     pnl_all.to_excel(writer, sheet_name="PnL")

#5. Sensitivity Analyses

Testing, how sensitive are the results regarding shifting the pattern within the 16 weeks cycle

In [36]:
# preparation of results dataframe, weekly data and setting parameters
result = pd.DataFrame(columns=['shift_wks','buyhold','strategy','strategy2'])
price_w2 = transforming_daily_weekly_open(price)
pos = pos_original
weeks = 16
for shift_weeks in range (weeks):
  # calculation of week-type and resulting positions depending on the shift
  pos_weekly(price_w2, pos, shift_weeks)
  pnl, res = pnl_calculation(price_w2)
  result.loc[len(result.index)] = [shift_weeks, res[0], res[1], res[2]]

In [39]:
# plotting results
fig = px.bar(result, x=result['shift_wks'], y=result['strategy'])
fig.update_layout(title='PnL for 16 weeks cycle (various shift parameters)')
fig.add_hline(y=res[0], line_dash='dot', line_color='green', annotation_text='Buy and Hold PnL', annotation_position='top left')
fig.update_layout(template = 'plotly_dark', autosize=False,width=800,height=400)
fig.update_layout(xaxis_title='shift parameter [in wks]', yaxis_title='PnL')